## A Guided Tour of LSST Calexps

This goal of this tutorial is to show best practices when working with LSST calexp objects.

The plan is to go through a visit image, and then to show how a coadd image differs.

**Owner:** David Shupe, Caltech/IPAC (@stargaser on Github, @dshupe in LSST Slack)

**Last run with:** lsst_distrib weekly version `lsst_w_2018_28`.

**Last run date:** 20 July 2018

## Imports

In [ ]:
from lsst.daf.persistence import Butler

In [ ]:
import lsst.afw.display as afw_display

### Retrieving and inspecting a calexp

For this tutorial we will use simulated LSST data.

Define a data directory and create a Butler

In [ ]:
datadir = '/project/shared/data/Twinkles_subset/output_data_v2'
butler = Butler(datadir)

Other notebooks *insert reference or link* show how to view what data are available in a Butler object. Here we get a specific one.

In [ ]:
dataId = {'filter': 'r', 'raft': '2,2', 'sensor': '1,1', 'visit': 235}
calexp = butler.get('calexp', **dataId)

In terms of pixel data, a calexp contains an image, a mask, and a variance.

Let's see how to access the image.

In [ ]:
calexp.image

To access the pixel values as an array, use the `.array` attribute.

In [ ]:
data = calexp.image.array
data

In [ ]:
data.__class__

Let's list all the methods for our calexp.

In [ ]:
calexp_methods = [m for m in dir(calexp) if not m.startswith('_')]

In [ ]:
calexp_methods

Access the masked Image

In [ ]:
calexp.maskedImage

Access the variance object and the underlying Numpy array

In [ ]:
calexp.variance

In [ ]:
calexp.variance.array

Access the mask and its underlying array

In [ ]:
calexp.mask

In [ ]:
calexp.mask.array

Get the dimensions of the image, mask and variance

In [ ]:
calexp.getDimensions()

The image, maskedImage and Exposure objects in `lsst.afw.display` include information on **LSST pixels**, which are 0-based with an optional offset.

For a calexp these are usually zero.

In [ ]:
calexp.getXY0()

In [ ]:
calexp.getX0(), calexp.getY0()

Access the wcs object

In [ ]:
wcs = calexp.getWcs()
wcs

The WCS object can be used e.g. to convert pixel coordinates into sky coordinates

In [ ]:
wcs.pixelToSky(100.0, 100.0)

Access the metadata

In [ ]:
metadata = calexp.getMetadata()

In [ ]:
metadata.getScalar('CCDTEMP')

Check if our calexp has a PSF

In [ ]:
calexp.hasPsf()

In [ ]:
psf = calexp.getPsf()

The PSF object can be used to get a realization of a PSF

In [ ]:
from lsst.geom.coordinates import Point2D
psfimage = psf.computeImage(Point2D(100.,100.))

Access the calibration object which can be used to convert instrumental magnitudes to AB magnitudes

In [ ]:
calib = calexp.getCalib()
calib

## Repeat for a coadd

In [ ]:
coadd_butler = Butler('/project/shared/data/with-globular/')

In [ ]:
coadd_butler.getKeys('deepCoadd_calexp')

We cannot use queryMetadata to look up what is available for coadds. This will be fixed in Butler Gen3.

For the time being, open a terminal and list files in `/project/shared/data/with-globular/`

In [ ]:
dataId = {'filter':'HSC-I', 'tract':9813, 'patch':'4,4'}

Retrieve a coadd

In [ ]:
coadd = coadd_butler.get('deepCoadd_calexp', dataId)

In [ ]:
coadd_methods = [m for m in dir(coadd) if not m.startswith('_')]

In [ ]:
coadd_methods

In [ ]:
set(coadd_methods).symmetric_difference(set(calexp_methods))

The result above shows that a calexp and a coadd have the same methods.

A coadd and a calexp differe mainly in the masks and the xy0 value.

In [ ]:
calexp.mask.getMaskPlaneDict()

In [ ]:
coadd.mask.getMaskPlaneDict()

In [ ]:
coadd.getXY0()